# Lesson 5: Conversational Question answering

![](./images/rag_diagram.png)

# Retrieval chain from previous lesson

Let's split and load our documents into a vector store and create a retriever. Then we will convert its output to a string.

In [3]:
import "dotenv/config";

[Module: null prototype] { default: {} }

In [4]:
import { loadAndSplitChunks } from "./lib/helpers.ts";

const splitDocs = await loadAndSplitChunks({
    chunkSize: 1536,
    chunkOverlap: 128
});

In [5]:
import { initializeVectorstoreWithDocuments } from "./lib/helpers.ts";

const vectorstore = await initializeVectorstoreWithDocuments({
  documents: splitDocs,
});

In [6]:
const retriever = vectorstore.asRetriever();

In [7]:
import { RunnableSequence } from "@langchain/core/runnables";
import { Document } from "@langchain/core/documents";

const convertDocsToString = (documents: Document[]): string => {
  return documents.map((document) => {
    return `<doc>\n${document.pageContent}\n</doc>`
  }).join("\n");
};

const documentRetrievalChain = RunnableSequence.from([
    (input) => input.question,
    retriever,
    convertDocsToString
]);

now that we have a retriever, lets build a retriever chain.

In [8]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const TEMPLATE_STRING = `You are an experienced researcher, 
expert at interpreting and answering questions based on provided sources.
Using the provided context, answer the user's question 
to the best of your ability using only the resources provided. 
Be verbose!

<context>

{context}

</context>

Now, answer this question using the above context:

{question}`;

const answerGenerationPrompt = ChatPromptTemplate.fromTemplate(
    TEMPLATE_STRING
);

In [9]:
import { ChatOpenAI } from "@langchain/openai";
import { StringOutputParser } from "@langchain/core/output_parsers";

const model = new ChatOpenAI({
    modelName: "gpt-3.5-turbo-1106"
});

In [10]:
const retrievalChain = RunnableSequence.from([
  {
    context: documentRetrievalChain,
    question: (input) => input.question,
  },
  answerGenerationPrompt,
  model,
  new StringOutputParser(),
]);

# Adding history

In [11]:
import { MessagesPlaceholder } from "@langchain/core/prompts";

const REPHRASE_QUESTION_SYSTEM_TEMPLATE = 
  `Given the following conversation and a follow up question, 
rephrase the follow up question to be a standalone question.`;

const rephraseQuestionChainPrompt = ChatPromptTemplate.fromMessages([
  ["system", REPHRASE_QUESTION_SYSTEM_TEMPLATE],
  new MessagesPlaceholder("history"),
  [
    "human", 
    "Rephrase the following question as a standalone question:\n{question}"
  ],
]);

In [12]:
const rephraseQuestionChain = RunnableSequence.from([
      rephraseQuestionChainPrompt,
      new ChatOpenAI({ temperature: 0.1, modelName: "gpt-3.5-turbo-1106" }),
      new StringOutputParser(),
])

In [13]:
import { HumanMessage, AIMessage } from "@langchain/core/messages";

const originalQuestion = "What are the prerequisites for this course?";

const originalAnswer = await retrievalChain.invoke({
  question: originalQuestion
});

console.log(originalAnswer);

The prerequisites for this course include familiarity with basic probability and statistics, as well as basic linear algebra. The instructor assumes that students are already familiar with concepts such as random variables, expectation, variance, and random variables. Additionally, knowledge of matrixes, vectors, matrix multiplication, matrix inverses, and eigenvectors is expected. It is mentioned that undergraduate statistics classes and linear algebra courses are generally sufficient for meeting these prerequisites. The instructor also suggests that knowing big O notation and having knowledge of data structures such as linked lists, queues, and binary trees is more important than specific programming languages like C or Java.


In [14]:
const chatHistory = [
  new HumanMessage(originalQuestion),
  new AIMessage(originalAnswer),
];

await rephraseQuestionChain.invoke({
  question: "Can you list them in bullet point form?",
  history: chatHistory,
});

"Could you please list the prerequisites for this course in bullet point form?"

# Putting it all together

In [15]:
const convertDocsToString = (documents: Document[]): string => {
  return documents.map((document) => `<doc>\n${document.pageContent}\n</doc>`).join("\n");
};

const documentRetrievalChain = RunnableSequence.from([
  (input) => input.standalone_question,
  retriever,
  convertDocsToString,
]);

In [16]:
const ANSWER_CHAIN_SYSTEM_TEMPLATE = `You are an experienced researcher, 
expert at interpreting and answering questions based on provided sources.
Using the below provided context and chat history, 
answer the user's question to the best of 
your ability 
using only the resources provided. Be verbose!

<context>
{context}
</context>`;

const answerGenerationChainPrompt = ChatPromptTemplate.fromMessages([
  ["system", ANSWER_CHAIN_SYSTEM_TEMPLATE],
  new MessagesPlaceholder("history"),
  [
    "human", 
    "Now, answer this question using the previous context and chat history:\n{standalone_question}"
  ]
]);

In [17]:
import { HumanMessage, AIMessage } from "@langchain/core/messages";
await answerGenerationChainPrompt.formatMessages({
  context: "fake retrieved content",
  standalone_question: "Why is the sky blue?",
  history: [
    new HumanMessage("How are you?"),
    new AIMessage("Fine, thank you!")
  ]
});

[
  SystemMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "You are an experienced researcher, \n" +
        "expert at interpreting and answering questions based on provided"... 210 more characters,
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "You are an experienced researcher, \n" +
      "expert at interpreting and answering questions based on provided"... 210 more characters,
    name: undefined,
    additional_kwargs: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "How are you?", additional_kwargs: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "How are you?",
    name: undefined,
    additional_kwargs: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "Fine, thank you!", additional_kwargs: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Fine, thank you!",
    name: undefined,
    additional_kwargs: {}
 

In [18]:
import { RunnablePassthrough } from "@langchain/core/runnables";

const conversationalRetrievalChain = RunnableSequence.from([
  RunnablePassthrough.assign({
    standalone_question: rephraseQuestionChain,
  }),
  RunnablePassthrough.assign({
    context: documentRetrievalChain,
  }),
  answerGenerationChainPrompt,
  new ChatOpenAI({ modelName: "gpt-3.5-turbo" }),
  new StringOutputParser(),
]);

In [19]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";

In [20]:
const messageHistory = new ChatMessageHistory();

const finalRetrievalChain = new RunnableWithMessageHistory({
  runnable: conversationalRetrievalChain,
  getMessageHistory: (_sessionId) => messageHistory,
  historyMessagesKey: "history",
  inputMessagesKey: "question",
});

In [21]:
const originalQuestion = "What are the prerequisites for this course?";

const originalAnswer = await finalRetrievalChain.invoke({
  question: originalQuestion,
}, {
  configurable: { sessionId: "test" }
});

const finalResult = await finalRetrievalChain.invoke({
  question: "Can you list them in bullet point form?",
}, {
  configurable: { sessionId: "test" }
});

console.log(finalResult);

Certainly! Here are the prerequisites for this course listed in bullet point form:

- Familiarity with basic probability and statistics, including random variables, expectation, variance, and random variable.
- Familiarity with basic linear algebra, including matrix and vector operations, matrix multiplication, matrix inverse, and eigenvectors of a matrix.
- Adequate programming skills, predominantly in MATLAB or Octave.
- No specific mention of prerequisites related to specific majors or prior coursework.

Please keep in mind that this answer is based solely on the provided context and chat history, and additional information from other sources or direct communication with the instructor may be necessary to obtain a comprehensive understanding of the course requirements.


https://smith.langchain.com/public/fca11abd-c0ec-456f-800e-6edfaf6bcf68/r